In [1]:
import pandas as pd

In [2]:
#Importando data de local
df = pd.read_csv('C:/Users/LENOVO/Downloads/employees_202308011354.csv')

In [25]:
df.columns

Index(['emp_no', 'birth_date', 'first_name', 'last_name', 'gender',
       'hire_date'],
      dtype='object')

In [6]:
#Importar datos generando una conexion con la base de datos
!pip install mysql-connector


     --------------------------------------- 11.9/11.9 MB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp39-cp39-win_amd64.whl size=247961 sha256=b5ec42e200cb82aed39a550c8fb9c6ed1885cd223b3c6636e4e6a97073c83628
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\7b\14\39\5aad423666e827dfe9a1fbcd111ac17171e7c9865d570780ce
Successfully built mysql-connector


In [84]:
import mysql.connector

connection = mysql.connector.connect(
 user= 'root',
 password = '',
 host= 'localhost',
 database= 'employees',
 #ssl_disabled=True
)

cursor = connection.cursor()

In [85]:
#connection.close()
#cursor.close()

True

In [14]:
query = """
    select *
    from employees e
    join salaries s 
    on e.emp_no = s.emp_no 
    where e.hire_date > '1999-01-01;'

"""

In [17]:
cursor.execute(query)

In [20]:
results = []
for i, data in enumerate(cursor):
    results.append(data)
    
cursor.close()
connection.close()  

In [22]:
len(results)

4852

In [26]:
df_pulled = pd.DataFrame(results)
df_pulled.columns = ['emp_no', 'birth_date', 'first_name', 'last_name', 'gender',
                     'hire_date', 'emp_no.1', 'salary', 'from_date', 'to_date']

df_pulled.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,emp_no.1,salary,from_date,to_date
0,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,46946,2000-04-29,2001-04-29
1,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,46775,2001-04-29,2002-04-29
2,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,50032,2002-04-29,9999-01-01
3,10105,1962-02-05,Hironoby,Piveteau,M,1999-03-23,10105,59258,1999-05-17,2000-05-16
4,10105,1962-02-05,Hironoby,Piveteau,M,1999-03-23,10105,61514,2000-05-16,2001-05-16


In [27]:
#Pandas para leer sql
#import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
print('sqlalchemy version', sqlalchemy.__version__)

sqlalchemy version 1.4.32


In [29]:
!pip install pymysql

     ---------------------------------------- 44.8/44.8 kB 1.1 MB/s eta 0:00:00


In [30]:
import pymysql

In [32]:
con_string = 'mysql+pymysql://root:#0@localhost/employees'
engine = create_engine(con_string)

query = """
    select *
    from employees e
    join salaries s 
    on e.emp_no = s.emp_no 
    where e.hire_date > '1999-01-01;'

"""
df_read_sql = pd.read_sql(query, engine)

In [33]:
df_read_sql

,emp_no,birth_date,first_name,last_name,gender,hire_date,emp_no,salary,from_date,to_date
0,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,44276,1999-04-30,2000-04-29
1,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,46946,2000-04-29,2001-04-29
2,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,46775,2001-04-29,2002-04-29
3,10019,1953-01-23,Lillian,Haddadi,M,1999-04-30,10019,50032,2002-04-29,9999-01-01
4,10105,1962-02-05,Hironoby,Piveteau,M,1999-03-23,10105,59258,1999-05-17,2000-05-16
...,...,...,...,...,...,...,...,...,...,...
4848,499610,1962-12-28,Kiyokazu,Vernadat,F,1999-08-29,499610,88013,2001-08-28,9999-01-01
4849,499888,1959-05-30,Hilari,Smeets,M,1999-06-27,499888,40000,1999-06-27,2000-06-26
4850,499888,1959-05-30,Hilari,Smeets,M,1999-06-27,499888,41159,2000-06-26,2001-06-26
4851,499888,1959-05-30,Hilari,Smeets,M,1999-06-27,499888,41709,2001-06-26,2002-06-26


In [34]:
#Escribiendo en la Base de datos

In [37]:
max_salary = df_pulled.groupby(['emp_no', 'first_name', 'last_name'])['salary'] \
            .max().reset_index()

In [38]:
max_salary

,emp_no,first_name,last_name,salary
0,10019,Lillian,Haddadi,50032
1,10105,Hironoby,Piveteau,61514
2,10298,Dietrich,DuCasse,43923
3,10684,Aimee,Tokunaga,47683
4,11315,Neven,Meriste,85452
...,...,...,...,...
1516,498990,Adil,Litecky,54960
1517,499361,Sandeepan,Solares,64196
1518,499553,Hideyuki,Delgrande,72320
1519,499610,Kiyokazu,Vernadat,88013


In [54]:
#Para guardar en base de datos.Si lo corremos dos veces nos dara error. Esto se setea con if_exists
max_salary.to_sql('max_salary', engine, if_exists='replace', index=False)

1521

In [55]:
max_salary

,emp_no,first_name,last_name,salary
0,10019,Lillian,Haddadi,50032
1,10105,Hironoby,Piveteau,61514
2,10298,Dietrich,DuCasse,43923
3,10684,Aimee,Tokunaga,47683
4,11315,Neven,Meriste,85452
...,...,...,...,...
1516,498990,Adil,Litecky,54960
1517,499361,Sandeepan,Solares,64196
1518,499553,Hideyuki,Delgrande,72320
1519,499610,Kiyokazu,Vernadat,88013


In [56]:
max_salary2 = pd.read_sql('SELECT * from max_salary', engine)

In [57]:
max_salary2

,emp_no,first_name,last_name,salary
0,10019,Lillian,Haddadi,50032
1,10105,Hironoby,Piveteau,61514
2,10298,Dietrich,DuCasse,43923
3,10684,Aimee,Tokunaga,47683
4,11315,Neven,Meriste,85452
...,...,...,...,...
1516,498990,Adil,Litecky,54960
1517,499361,Sandeepan,Solares,64196
1518,499553,Hideyuki,Delgrande,72320
1519,499610,Kiyokazu,Vernadat,88013


In [58]:
max_salary2.shape, max_salary.shape

((1521, 4), (1521, 4))

In [60]:
#to_sql appending a table. Agregar datos a la tabla. Columna create_date

In [73]:
max_salary_plus = max_salary.copy()
max_salary_plus['salary'] = max_salary_plus['salary'] + 1000000

In [74]:
import datetime as dt

In [75]:
max_salary_plus.to_sql('max_salary', engine, index=False, if_exists='replace')

1521

In [80]:
max_salary['create_date'] = dt.datetime.now()
max_salary.to_sql('max_salary', engine, index=False, if_exists='append')

1521

In [81]:
max_salary3 = pd.read_sql('SELECT * from max_salary', engine)

In [82]:
max_salary3['create_date'].value_counts()

2023-08-01 15:20:22    1521
2023-08-01 15:21:21    1521
2023-08-01 15:24:24    1521
Name: create_date, dtype: int64